In [59]:
# Eduardo Velasco R.
# Creado el 22 de enero de 2023
# Script de reducción de dimensionalidad manual 
# El script de replica actualizado con datos del 2023 puede estar integrado, pero se encuentra a parte como 'HistoricCovid_2023'


# Recordar que las fases 3-6 de la metodología BFMDS son:   # 3. Requisitos de datos
                                                            # 4. Recopilación de datos
                                                            # 5. Comprensión de datos
                                                            # 6. Preparación de datos

# FASE DE COMPRENSIÓN DE DATOS

# ¿Tienen sentido los datos a simple vista?
#   Comprobar fuente, descargar nuevamente, revisar si existe algún tipo de encriptación
# Revisar descripción de las variables 
# Identificar variables de interés o desinterés y registrarlas (DEBO HACER ESTO ENTENDIENDO LA INTENCIONALIDAD DE CADA DATASET)
# ¿En qué formato están los datos?
#   - ¿Debo cambiarlo?
#     - ¿Cómo lo cambio?
# ¿Es necesario identificar el tipo de variable de cada atributo (str, int, bool, etc.)?
# En caso de que sea una base de datos (varios datasets)
#   - ¿Cuáles son los identificadores externos?

# FASE DE PREPAREACIÓN DE DATOS

# Para cada conjunto de datos
# Con base en la comprensión de datos efectúo las tareas de limpieza de datos (Corrección, adición, eliminación)
# ¿Qué atributos me sirven? (Filtro vertical)
# ¿Qué restricciones requiero aplicar a cada atributo? (Filtro horizontal)
# ¿Es necesario guardar los cambios en un dataset nuevo para agilizar los futuros pasos? 
#   - (Creación de DataWarehouse intermedio) --> Podría compararse el tiempo invertido entre usarlo o no
#     Dentro de Perez2015_DataPrep se incorporaría después de GDP y antes de SDP

In [131]:
# Importo las librerías que utilizaré, si en el camino necesito más las añado
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [132]:
# Considerar poner todas las funciones aquí

# DEFINO LAS FUNCIONES DE FILTRADO VERTICAL Y HORIZONTAL
# Filtro vertical
# dframe: el dataframe de interés, valores: los atributos que me interesa conservar
def vert_filt(dframe, valores):
    nombre = dframe[valores]
    print('Número de atributos del dataframe: ' + str(len(valores)) + '; cada uno con ' + str(len(nombre)) + ' datos.')
    return nombre

# Fitro horizontal
# dframe: el dataframe de interés, var: atributo, restriccion: tipo de restriccion (=, <, >) y valor: valor de la restricción
def hor_filt(dframe, var, restriccion, valor):
    if restriccion == '=':
        indices = dframe[dframe[var] == valor].index
        nuevo = dframe.drop(indices)
        print('Longitud nueva del dataframe: '+str(len(nuevo))+'. Filtro de '+str(var)+str(restriccion)+str(valor)+' aplicado.')
    if restriccion == '>':
        indices = dframe[dframe[var] > valor].index
        nuevo = dframe.drop(indices)
        print('Longitud nueva del dataframe: '+str(len(nuevo))+'. Filtro de '+str(var)+str(restriccion)+str(valor)+' aplicado.')
    if restriccion == '<':
        indices = dframe[dframe[var] < valor].index
        nuevo = dframe.drop(indices)
        print('Longitud nueva del dataframe: '+str(len(nuevo))+'. Filtro de '+str(var)+str(restriccion)+str(valor)+' aplicado.')
    return nuevo

# Fitro horizontal por rango
# dframe: el dataframe de interés, var: atributo, minval: valor inicial, maxval: valor final
def hor_f_rang(dframe, var, minval, maxval):
    indices = dframe[dframe[var] < minval].index
    nuevo = dframe.drop(indices)
    indicesm = nuevo[nuevo[var] > maxval].index
    nuevom = nuevo.drop(indicesm)
    print('Longitud nueva del dataframe: '+str(len(nuevom))+'. Desde '+str(var)+': '+str(minval)+' hasta '+str(maxval))
    return nuevom

def vert_filt2(dframe,atributo, claves):
    nuevo = dframe[dframe[atributo].isin(claves)]
    return nuevo

# defino función de identificadores únicos
def gen_id(dframe, atributo1, atributo2):
    claves_ent = list(dframe[atributo1].values)
    claves_mun = list(dframe[atributo2].values)  
    indices = []
    for j in range(0, len(dframe)):
        id1 = str(claves_ent[j])
        id2 = str(claves_mun[j])
        ID_pob = ''
        lng2 = len(id2)
        if lng2 == 3:
            ID_pob = id1 + id2
            indices.append(ID_pob)
        if lng2 == 2:
            ID_pob = id1 + '0' + id2
            indices.append(ID_pob)
        if lng2 == 1:
            ID_pob = id1 + '00' + id2
            indices.append(ID_pob)
    return indices

# defino función de fechas mes dia
def mes_dia(dframe,atributo0, atributo1, atributo2):
    claves_ani = list(dframe[atributo0].values)
    claves_mes = list(dframe[atributo1].values)
    claves_dia = list(dframe[atributo2].values)  
    indices = []
    for j in range(0, len(dframe)):
        id0 = str(claves_ani[j])
        id1 = str(claves_mes[j])
        id2 = str(claves_dia[j])
        ID_pob = ''
        if len(id1) == 1:      # 2 de feb: 2/2
            if len(id2) == 1:
                ID_pob = id0 + '0' + id1 + '0' + id2
                indices.append(int(ID_pob))
            else:              # 12 de feb
                ID_pob = id0 + '0' + id1 + id2
                indices.append(int(ID_pob))
        else:
            if len(id2) == 1:
                ID_pob = id0 + id1 + '0' + id2
                indices.append(int(ID_pob))
            else:
                ID_pob = id0 + id1 + id2
                indices.append(int(ID_pob))
    return indices

def acople_col(dframe, lista, nombre): # Se solicita el dataframe, la lista de elementos por añadir y el nombre de la columna
    loc_n = dframe.shape[1]
    dframe.insert(
        loc = loc_n,         # Podría buscarse una forma de que lo agregara al final de la lista
        column = nombre,
        value = lista,
        allow_duplicates = False, # False restringe la duplicación de columnas
    )
    
# Funición de filtrado horizontal por claves de enfermedades 
# se puede usar para los datos de claves de AGEE 
def cve_enf(dframe, claves):
    nuevo = dframe[dframe['causa_def'].isin(claves)]
    return nuevo

# mFunción filtrado de municipios de interés (muy similar a la función de arriba)
def cve_agee(dframe, claves):
    nuevo = dframe[dframe['Id_mun_agee'].isin(claves)]
    return nuevo

# Función de conteo de enfermedades
def cont_enf(id_235, id_dgis):
    enfermedades = []
    for i in range(0, len(id_235)): # idnetificadores de los 233
        cont = 0
        for j in range(0, len(id_dgis)): # identificadores todos
            if id_235[i] == id_dgis[j]:
                cont = cont + 1
        enfermedades.append(cont)
    return enfermedades

# Densidad de población
def den_pob(dframe):  # dframe es donde vienen los datos de km y de poblacion
                      # esta función produce que SanQuintín quede en cero en dens_pobl   
    lista_pobtot = list(dframe['POBTOT'].values)
    km2 = list(dframe['km2'].values) 
    lista_densidad = []
    for i in range(0, len(km2)):   
        if km2[i] == 0:
            den = 0
        else:
            den = round(lista_pobtot[i]/km2[i],5)   
        lista_densidad.append(den)
    return lista_densidad

# Tasa de mortalidad
def tasa_mort(dframe, id_muertes):
    lista_pobtot = list(dframe['POBTOT'].values) 
    muertes_cov = list(dframe[id_muertes].values) 
    lista_mort = []
    for i in range(0, len(dframe)):
        tasa = 0
        tasa = round((muertes_cov[i]/lista_pobtot[i])*100000, 5)   # muertes / población * 100000  
        lista_mort.append(tasa)
    return lista_mort

def tasa_mort21(dframe):  # es igualita a la anterior pero la necesito para los de 2021
    lista_pobtot = list(dframe['POBTOT'].values) 
    muertes_cov = list(dframe['total_muertes_21'].values) 
    lista_mort = []
    for i in range(0, len(dframe)):
        tasa = 0
        tasa = round((muertes_cov[i]/lista_pobtot[i])*100000, 5)   # muertes / población * 100000  
        lista_mort.append(tasa)
    return lista_mort

# Normalización de atributos
def normm(dframe, atributo):
    lista = list(dframe[atributo].values)
    list_norm = []
    maxi = max(lista)
    mini = min(lista)
    for i in range(0, len(lista)):
        valorN = (lista[i]-mini)/(maxi-mini)
        list_norm.append(round(valorN,5))
    return list_norm

In [133]:
def tipo(dframe):
    for column in dframe:
        print(column + ' es de tipo ' + str(type(dframe[column][1])))

In [134]:
## Importo los conjuntos de datos ##
# Dataset DGIS                                               # PARA ESTE SI HAY DATOS DEL 2020 Y 2021 
DGIS_defnsn = pd.read_csv(r'DGIS_defunsiones.csv')           # a simple vista parece que los atributos no cambian
df_DGIS = pd.DataFrame(DGIS_defnsn)
#print(df_DGIS.head())
DGIS_defnsn21 = pd.read_csv(r'DGIS_defunsiones_2021.csv')    # Aplicar lo mismo que para el 2020
df_DGIS21 = pd.DataFrame(DGIS_defnsn21)
print(df_DGIS21.head())

   ent_regis  mun_regis  ent_resid  mun_resid  tloc_resid  loc_resid  \
0          1          1          1          1          15          1   
1          1          1          1          1          15          1   
2          1          1          1          6           9          1   
3          1          1          1          1          15          1   
4          1          1          1          2           5          1   

   ent_ocurr  mun_ocurr  tloc_ocurr  loc_ocurr  ... anio_cert maternas  \
0          1          1          15          1  ...      2021      NaN   
1          1          1          15          1  ...      2021      NaN   
2          1          6           9          1  ...      2021      NaN   
3          1          1          15          1  ...      2021      NaN   
4          1          2           5          1  ...      2021      NaN   

   lengua  cond_act  par_agre  ent_ocules  mun_ocules  loc_ocules  razon_m  \
0       2         2        88          88   

In [135]:
# Dataset INEGI   
INEGI_population = pd.read_csv(r'INEGI_population.csv', low_memory=False)
df_INEGI = pd.DataFrame(INEGI_population)
#print(df_INEGI.head())

In [136]:
# Dataset AGEE  
AGEE_spatial = pd.read_csv(r'AGEE_geostatistical.csv', encoding='latin', low_memory=False)
df_AGEE = pd.DataFrame(AGEE_spatial)
print(df_AGEE.head())

       Mapa  Cve_Ent         Nom_Ent Nom_Abr  Cve_Mun         Nom_Mun  \
0  10010001        1  Aguascalientes    Ags.        1  Aguascalientes   
1  10010094        1  Aguascalientes    Ags.        1  Aguascalientes   
2  10010096        1  Aguascalientes    Ags.        1  Aguascalientes   
3  10010100        1  Aguascalientes    Ags.        1  Aguascalientes   
4  10010102        1  Aguascalientes    Ags.        1  Aguascalientes   

   Cve_Loc                 Nom_Loc Ámbito         Latitud         Longitud  \
0        1          Aguascalientes      U  21°52´47.362N"  102°17´45.768W"   
1       94          Granja Adelita      R  21°52´18.749N"  102°22´24.710W"   
2       96               Agua Azul      R  21°53´01.522N"  102°21´25.639W"   
3      100           Rancho Alegre      R  21°51´16.556N"  102°22´21.884W"   
4      102  Los Arbolitos [Rancho]      R  21°46´48.650N"  102°21´26.261W"   

   Lat_Decimal  Lon_Decimal Altitud Cve_Carta Pob_Total Pob_Masculina  \
0    21.879823  -10

In [137]:
# Aplico filtro vertical para el dataset de INEGI

tabla1 = vert_filt(df_INEGI, ['ENTIDAD', 'NOM_ENT', 'MUN', 'NOM_MUN', 'POBTOT', 'LOC'])
#print(tabla1)

Número de atributos del dataframe: 6; cada uno con 195662 datos.


In [138]:
# Aplico filtros horizontales, debe hacerse uno por uno

nuevo1=hor_filt(tabla1, 'MUN', '=',0) # Quito los estados repetidos, para esto tuve que revisar el dataset
#print(nuevo1)
nuevo2=hor_filt(nuevo1, 'POBTOT', '<',100000)    # Quito la población con menos de 100k habitantes
#print(nuevo2)
inegi_vh=hor_filt(nuevo2, 'LOC', '>',0) # Sólo me sirven los valores con LOC = 0, ya que son los que ofrecen la información
                                      # de todo el municipio en general
print(inegi_vh)
# Podría reescribirse en una sola variable para ahorrar memoria pero lo hago en distintas para ver su comportamiento

Longitud nueva del dataframe: 195563. Filtro de MUN=0 aplicado.
Longitud nueva del dataframe: 380. Filtro de POBTOT<100000 aplicado.
Longitud nueva del dataframe: 235. Filtro de LOC>0 aplicado.
        ENTIDAD          NOM_ENT  MUN         NOM_MUN   POBTOT  LOC
6             1   Aguascalientes    1  Aguascalientes   948990    0
985           1   Aguascalientes    5     Jesús María   129929    0
2064          2  Baja California    1        Ensenada   443807    0
3755          2  Baja California    2        Mexicali  1049792    0
5728          2  Baja California    3          Tecate   108440    0
...         ...              ...  ...             ...      ...  ...
189008       31          Yucatán   41         Kanasín   141939    0
189128       31          Yucatán   50          Mérida   995129    0
191420       32        Zacatecas   10       Fresnillo   240532    0
192240       32        Zacatecas   17       Guadalupe   211740    0
195542       32        Zacatecas   56       Zacatecas   14

In [139]:
# Paso a primer CSV para guardar en primer dataWarehouse 
# inegi_vh.to_csv('DW_1_estadosfiltrados.csv', index=False)

In [140]:
# Filtro vertical y horizontalmente para los datos de AGEE
# Necesito quedarme con las claves también para filtrar los municipios que me interesan
agee_v = vert_filt(df_AGEE, ['Cve_Ent', 'Cve_Mun', 'Lat_Decimal', 'Lon_Decimal', 'Altitud', 'Cve_Loc', 'Pob_Total'])
agee_vh = hor_filt(agee_v, 'Cve_Loc', '>', 1) # Para esto revisé previamente el dataset, dándome cuenta que únicamente
                                              # necesito las localidades con clave igual a 1 y desecho lo demás ya que
                                              # en este caso son los valores de cada localidad del municipio
# debe tener la longitud del número total de municipios, aprox 2473

Número de atributos del dataframe: 7; cada uno con 300163 datos.
Longitud nueva del dataframe: 2473. Filtro de Cve_Loc>1 aplicado.


In [141]:
# paso a csv el dataFrame anterior para un segundo dataWarehouse intermedio
# agee_vh.to_csv('DW_1_geograficos.csv', index=False) 

# ESTOS PASOS NO DEBERÍAN INGNORARSE PORQUE HARÍA QUE SE EJECUTARA TODO MÁS RÁPIDO

In [142]:
ids_agee = gen_id(agee_vh, 'Cve_Ent', 'Cve_Mun')  # El nombre de esta variable la usaré más adelante para conteo
print(len(ids_agee))

2473


In [143]:
# Aplico la función de generación de identificadores
identificadores = gen_id(inegi_vh,'ENTIDAD', 'MUN')  # El nombre de esta variable la usaré más adelante para conteo\n",
print(len(identificadores)) # es una lista, no olvidar

235


In [144]:
acople_col(agee_vh, ids_agee, 'Id_mun_agee')
print(agee_vh)

        Cve_Ent  Cve_Mun  Lat_Decimal  Lon_Decimal Altitud  Cve_Loc Pob_Total  \
0             1        1    21.879823  -102.296047    1878        1    863893   
717           1        2    22.238317  -102.089275    2167        1      5248   
959           1        3    21.846907  -102.718751    1634        1     21049   
1250          1        4    22.366409  -102.300044    2005        1      5870   
1344          1        5    21.961273  -102.343416    1877        1     63805   
...         ...      ...          ...          ...     ...      ...       ...   
299753       32       54    22.357088  -101.712599    2161        1      4851   
299814       32       55    22.354259  -102.883726    1900        1     13641   
299967       32       56    22.776096  -102.571836    2427        1    138444   
300079       32       57    22.735389  -102.366038    2193        1     17021   
300121       32       58    21.512178  -103.407042    1921        1      1995   

       Id_mun_agee  
0     

In [145]:
acople_col(inegi_vh, identificadores, 'Id_mun') # Compilar esta línea más de una vez debe arrojar un error
print(inegi_vh)                                 # hecho lo anterior ya podría quitar los atributos de LOC, ENTIDAD y MUN
inegi_vh = vert_filt(inegi_vh, ['NOM_MUN', 'POBTOT', 'Id_mun'])   
print(inegi_vh)
nombres_mun = vert_filt(inegi_vh, ['NOM_MUN'])
pob_tot = vert_filt(inegi_vh, ['POBTOT'])

        ENTIDAD          NOM_ENT  MUN         NOM_MUN   POBTOT  LOC Id_mun
6             1   Aguascalientes    1  Aguascalientes   948990    0   1001
985           1   Aguascalientes    5     Jesús María   129929    0   1005
2064          2  Baja California    1        Ensenada   443807    0   2001
3755          2  Baja California    2        Mexicali  1049792    0   2002
5728          2  Baja California    3          Tecate   108440    0   2003
...         ...              ...  ...             ...      ...  ...    ...
189008       31          Yucatán   41         Kanasín   141939    0  31041
189128       31          Yucatán   50          Mérida   995129    0  31050
191420       32        Zacatecas   10       Fresnillo   240532    0  32010
192240       32        Zacatecas   17       Guadalupe   211740    0  32017
195542       32        Zacatecas   56       Zacatecas   149607    0  32056

[235 rows x 7 columns]
Número de atributos del dataframe: 3; cada uno con 235 datos.
              

In [146]:
# Filtro los muns de agee
agee_muns = cve_agee(agee_vh, identificadores) # Función que sólo deja los 235 muns
print(agee_muns)  # Deben ser los 235 municipios filtrados

        Cve_Ent  Cve_Mun  Lat_Decimal  Lon_Decimal Altitud  Cve_Loc Pob_Total  \
0             1        1    21.879823  -102.296047    1878        1    863893   
1344          1        5    21.961273  -102.343416    1877        1     63805   
2904          2        1    31.808944  -116.595134      18        1    330652   
5355          2        2    32.641176  -115.475579       0        1    854186   
7664          2        3    32.572679  -116.626550     526        1     81059   
...         ...      ...          ...          ...     ...      ...       ...   
286211       31       41    20.934640   -89.558059       9        1    139753   
286682       31       50    20.967537   -89.624392      10        1    921771   
294143       32       10    23.174895  -102.868229    2180        1    143281   
295298       32       17    22.746782  -102.518802    2279        1    170029   
299967       32       56    22.776096  -102.571836    2427        1    138444   

       Id_mun_agee  
0     

In [147]:
datos_agee = vert_filt(agee_muns, ['Lat_Decimal', 'Lon_Decimal', 'Altitud', 'Id_mun_agee'])
print(datos_agee.head())

Número de atributos del dataframe: 4; cada uno con 235 datos.
      Lat_Decimal  Lon_Decimal Altitud Id_mun_agee
0       21.879823  -102.296047    1878        1001
1344    21.961273  -102.343416    1877        1005
2904    31.808944  -116.595134      18        2001
5355    32.641176  -115.475579       0        2002
7664    32.572679  -116.626550     526        2003


In [148]:
list_muns = list(nombres_mun['NOM_MUN'])  # Existen formas más rapidas de hacer esto
list_pob = list(pob_tot['POBTOT'])
#print(list_muns)

In [149]:
# Agrego los nombres de los municipios
acople_col(datos_agee, list_muns, 'NOM_MUN')

In [150]:
print(datos_agee)

        Lat_Decimal  Lon_Decimal Altitud Id_mun_agee         NOM_MUN
0         21.879823  -102.296047    1878        1001  Aguascalientes
1344      21.961273  -102.343416    1877        1005     Jesús María
2904      31.808944  -116.595134      18        2001        Ensenada
5355      32.641176  -115.475579       0        2002        Mexicali
7664      32.572679  -116.626550     526        2003          Tecate
...             ...          ...     ...         ...             ...
286211    20.934640   -89.558059       9       31041         Kanasín
286682    20.967537   -89.624392      10       31050          Mérida
294143    23.174895  -102.868229    2180       32010       Fresnillo
295298    22.746782  -102.518802    2279       32017       Guadalupe
299967    22.776096  -102.571836    2427       32056       Zacatecas

[235 rows x 5 columns]


In [151]:
# paso a csv el dataFrame anterior para un TERCER dataWarehouse de mayor utilidad 
# datos_agee.to_csv('DW_2_geograficos.csv', index=False)

In [152]:
# HASTA AQUÍ ACABA LO RESPECTIVO A LOS MUNICIPIOS, LO QUE SIGUE SON LAS ENFERMEDADES

In [153]:
# Identificadores DGIS
DGIS_ids_regis = gen_id(df_DGIS,'ent_regis', 'mun_regis')  # Ambos identificadores
DGIS_ids_regis21 = gen_id(df_DGIS21,'ent_regis', 'mun_regis')
DGIS_ids_resid = gen_id(df_DGIS,'ent_resid', 'mun_resid')  # De _regis a _resid
DGIS_ids_resid21 = gen_id(df_DGIS21,'ent_resid', 'mun_resid')  # Usaré esta variable más adelante
print(DGIS_ids_regis[808210:808215])   # sirve para DGIS
print(DGIS_ids_resid21[808210:808215])

['21114', '21114', '21020', '21208', '21114']
['21115', '21114', '21001', '21114', '21054']


In [154]:
# Acoplo identificadores a DEGIS
acople_col(df_DGIS, DGIS_ids_regis, 'Id_mun_DGIS_regis')
acople_col(df_DGIS21, DGIS_ids_regis21, 'Id_mun_DGIS_regis')
acople_col(df_DGIS, DGIS_ids_resid, 'Id_mun_DGIS_resid')
acople_col(df_DGIS21, DGIS_ids_resid21, 'Id_mun_DGIS_resid')

In [155]:
# Para el conteo de enfermedades es necesario obtener la información que describa lo que nos interesa (Covid en este caso)

In [156]:
# Llamo a la función de filtrado horizontal de claves de enfermedades
# Hay dos opciones para este punto respecto a las enfermedades:
    # Conteo de enfermedades para todo México
    # Conteo sólo para las entidades filtradas 

claves = ['U071', 'U072', 'U099', 'U109']
muertes_df = cve_enf(df_DGIS, ['U071', 'U072', 'U099', 'U109'])
muertes_df21 = cve_enf(df_DGIS21, ['U071', 'U072', 'U099', 'U109'])
# Hago dataframe con los atributos que me interesan
tabla_muertes = vert_filt(muertes_df, ['ent_resid', 'mun_resid', 'causa_def', 'Id_mun_DGIS_regis', 'Id_mun_DGIS_resid','dia_regis', 'mes_regis', 'anio_regis'])
tabla_muertes21 = vert_filt(muertes_df21, ['ent_resid', 'mun_resid', 'causa_def', 'Id_mun_DGIS_regis', 'Id_mun_DGIS_resid', 'dia_regis', 'mes_regis', 'anio_regis'])
print(tabla_muertes.head())
print(len(tabla_muertes))
print(tabla_muertes21.head())
print(len(tabla_muertes21))

# Este dataFrame sólo es informativo y para contar el número de muertes, no se ocupa para otra cosa

# Paso a csv primera tabla de agee
#tabla_muertes_1.to_csv('emce_1.csv', index=False) 

Número de atributos del dataframe: 8; cada uno con 200270 datos.
Número de atributos del dataframe: 8; cada uno con 238772 datos.
     ent_resid  mun_resid causa_def Id_mun_DGIS_regis Id_mun_DGIS_resid  \
5            1          1      U071              1001              1001   
19           1          1      U071              1001              1001   
585          1          1      U071              1001              1001   
627          1          1      U071              1001              1001   
683          1         11      U071              1001              1011   

     dia_regis  mes_regis  anio_regis  
5           29         10        2020  
19          13         11        2020  
585         29          4        2020  
627         18          5        2020  
683         26          5        2020  
200270
    ent_resid  mun_resid causa_def Id_mun_DGIS_regis Id_mun_DGIS_resid  \
56          1          1      U072              1001              1001   
60          1          1

In [157]:
# Saco identificadores de fecha
ids_fecha20 = mes_dia(tabla_muertes, 'anio_regis', 'mes_regis', 'dia_regis')
#print(ids_fecha20)
ids_fecha21 = mes_dia(tabla_muertes21, 'anio_regis', 'mes_regis', 'dia_regis')
#print(ids_fecha21)

In [158]:
# Acoplo 
acople_col(tabla_muertes, ids_fecha20, 'fecha_regis')
acople_col(tabla_muertes21, ids_fecha21, 'fecha_regis')

In [159]:
# Filtro
tabla_muertes = vert_filt(tabla_muertes, ['ent_resid', 'mun_resid', 'causa_def', 'Id_mun_DGIS_regis', 'Id_mun_DGIS_resid','fecha_regis'])
tabla_muertes21 = vert_filt(tabla_muertes21, ['ent_resid', 'mun_resid', 'causa_def', 'Id_mun_DGIS_regis', 'Id_mun_DGIS_resid','fecha_regis'])

Número de atributos del dataframe: 6; cada uno con 200270 datos.
Número de atributos del dataframe: 6; cada uno con 238772 datos.


In [128]:
######################################################################
#####                                                            #####
#####    El Acotamiento de fechas debe hacerse por esta zona     #####
#####                                                            #####
######################################################################

# Copias
tabla_muerts20_1 = tabla_muertes
tabla_muerts20_2 = tabla_muertes
tabla_muerts21_1 = tabla_muertes21
tabla_muerts21_2 = tabla_muertes21
print(len(tabla_muerts20_1))
print(len(tabla_muerts21_1))

200270
238722


In [176]:
tabla_muertes20 = hor_f_rang(tabla_muertes, 'fecha_regis', 20200209, 20201299)      # 2020
tabla_muertes21 = hor_f_rang(tabla_muertes21, 'fecha_regis', 20210101, 20211299)    # 2021
# Divisiones de tiempo
tabla_muertes20_1 = hor_f_rang(tabla_muerts20_1, 'fecha_regis', 20200209, 20200799) # primera mitad 2020
tabla_muertes20_2 = hor_f_rang(tabla_muerts20_2, 'fecha_regis', 20200801, 20201299) # segunda mitad 2020 
tabla_muertes21_1 = hor_f_rang(tabla_muerts21_1, 'fecha_regis', 20210101, 20210699) # primera mitad 2021
tabla_muertes21_2 = hor_f_rang(tabla_muerts21_2, 'fecha_regis', 20210701, 20211299) # segunda mitad 2021  
#print(tabla_muertes20_1)
#print(tabla_muertes_prueba)  # Funciona!

Longitud nueva del dataframe: 200270. Desde fecha_regis: 20200209 hasta 20201299
Longitud nueva del dataframe: 238722. Desde fecha_regis: 20210101 hasta 20211299
Longitud nueva del dataframe: 81829. Desde fecha_regis: 20200209 hasta 20200799
Longitud nueva del dataframe: 118441. Desde fecha_regis: 20200801 hasta 20201299
Longitud nueva del dataframe: 139820. Desde fecha_regis: 20210101 hasta 20210699
Longitud nueva del dataframe: 98902. Desde fecha_regis: 20210701 hasta 20211299


In [177]:
lista20_1 = list(tabla_muertes20_1['fecha_regis'])
lista20_1.sort()
lista20_2 = list(tabla_muertes20_2['fecha_regis'])
lista20_2.sort()
print(len(lista20_1))
print(len(lista20_2))
#toda
lista20_ = list(tabla_muertes20['fecha_regis'])
lista20_.sort()
#print(lista20_)

81829
118441


In [178]:
# Lista de los 235 muns y sus muertes

identificadores2 = list(inegi_vh['Id_mun'].values) # 235 municipios filtrados

# AÑO 2020
#idents_regis = list(tabla_muertes20['Id_mun_DGIS_regis'].values) # 
#idents_resid = list(tabla_muertes20['Id_mun_DGIS_resid'].values) # 

# Primera mitad
idents_regis_20_1 = list(tabla_muertes20_1['Id_mun_DGIS_regis'].values) # 
idents_resid_20_1 = list(tabla_muertes20_1['Id_mun_DGIS_resid'].values) #

# Segunda mitad
idents_regis_20_2 = list(tabla_muertes20_2['Id_mun_DGIS_regis'].values) # 
idents_resid_20_2 = list(tabla_muertes20_2['Id_mun_DGIS_resid'].values) #

lista_covid_regis_20_1 = cont_enf(identificadores2, idents_regis_20_1)
lista_covid_resid_20_1 = cont_enf(identificadores2, idents_resid_20_1)

lista_covid_regis_20_2 = cont_enf(identificadores2, idents_regis_20_2)
lista_covid_resid_20_2 = cont_enf(identificadores2, idents_resid_20_2)

print(len(lista_covid_regis_20_1))
print(len(lista_covid_resid_20_1))

print(len(lista_covid_regis_20_2))
print(len(lista_covid_resid_20_2))

235
235
235
235


In [179]:
# AÑO 2021
#idents_regis = list(tabla_muertes21['Id_mun_DGIS_regis'].values) # 
#idents_resid = list(tabla_muertes21['Id_mun_DGIS_resid'].values) #

# Primera mitad
idents_regis_21_1 = list(tabla_muertes21_1['Id_mun_DGIS_regis'].values) # 
idents_resid_21_1 = list(tabla_muertes21_1['Id_mun_DGIS_resid'].values) #

# Segunda mitad
idents_regis_21_2 = list(tabla_muertes21_2['Id_mun_DGIS_regis'].values) # 
idents_resid_21_2 = list(tabla_muertes21_2['Id_mun_DGIS_resid'].values) #

lista_covid_regis_21_1 = cont_enf(identificadores2, idents_regis_21_1)
lista_covid_resid_21_1 = cont_enf(identificadores2, idents_resid_21_1)

lista_covid_regis_21_2 = cont_enf(identificadores2, idents_regis_21_2)
lista_covid_resid_21_2 = cont_enf(identificadores2, idents_resid_21_2)

print(len(lista_covid_regis_21_1))
print(len(lista_covid_resid_21_1))

print(len(lista_covid_regis_21_2))
print(len(lista_covid_resid_21_2))

235
235
235
235


In [185]:
# Para el año 2020
#print(lista_covid_regis_20_1)
#print(lista_covid_resid_20_1)
print('Suma primera mitad 2020 datos de registro:  ',sum(lista_covid_regis_20_1)) # la suma debe ser 177147 porque ya son los municipios filtrados
print('Suma primera mitad 2020 datos de residencia:',sum(lista_covid_resid_20_1)) # suma: 158121
print('Suma segunda mitad 2020 datos de registro:  ',sum(lista_covid_regis_20_2)) # 195525
print('Suma segunda mitad 2020 datos de residencia:',sum(lista_covid_resid_20_2)) # 172469

# Para el año 2021
#print(lista_covid_regis_21_1)
#print(lista_covid_resid_21_1)
print('Suma primera mitad 2021 datos de registro:  ',sum(lista_covid_regis_21_1)) # 195525
print('Suma primera mitad 2021 datos de residencia:',sum(lista_covid_resid_21_1)) # 172469 
print('Suma segunda mitad 2021 datos de registro:  ',sum(lista_covid_regis_21_2)) # 195525
print('Suma segunda mitad 2021 datos de residencia:',sum(lista_covid_resid_21_2)) # 172469
# CONCLUSIÓN: USAR LOS DATOS DE RESIDENCIA, PERO NO SE DESCARTA COMPARAR AMBOS

Suma primera mitad 2020 datos de registro:   73602
Suma primera mitad 2020 datos de residencia: 65938
Suma segunda mitad 2020 datos de registro:   103545
Suma segunda mitad 2020 datos de residencia: 92183
Suma primera mitad 2021 datos de registro:   119101
Suma primera mitad 2021 datos de residencia: 107763
Suma segunda mitad 2021 datos de registro:   76382
Suma segunda mitad 2021 datos de residencia: 64689


In [186]:
# Debo sacar el número de muertes por munucipio y después integrar los datos al df de inegi o agee

#acople_col(datos_agee, lista_covid_regis, 'muertes_regis_20')
#acople_col(datos_agee, lista_covid_regis21, 'muertes_regis_21')
#acople_col(datos_agee, lista_covid_resid, 'muertes_resid_20')
#acople_col(datos_agee, lista_covid_resid21, 'muertes_resid_21')

# MITADES
acople_col(datos_agee, lista_covid_regis_20_1, 'muertes_regis_20_1') # 2020
acople_col(datos_agee, lista_covid_regis_20_2, 'muertes_regis_20_2')
acople_col(datos_agee, lista_covid_resid_20_1, 'muertes_resid_20_1')
acople_col(datos_agee, lista_covid_resid_20_2, 'muertes_resid_20_2')
acople_col(datos_agee, lista_covid_regis_21_1, 'muertes_regis_21_1') # 2021
acople_col(datos_agee, lista_covid_regis_21_2, 'muertes_regis_21_2')
acople_col(datos_agee, lista_covid_resid_21_1, 'muertes_resid_21_1')
acople_col(datos_agee, lista_covid_resid_21_2, 'muertes_resid_21_2')

In [187]:
print(datos_agee)

        Lat_Decimal  Lon_Decimal Altitud Id_mun_agee         NOM_MUN  \
0         21.879823  -102.296047    1878        1001  Aguascalientes   
1344      21.961273  -102.343416    1877        1005     Jesús María   
2904      31.808944  -116.595134      18        2001        Ensenada   
5355      32.641176  -115.475579       0        2002        Mexicali   
7664      32.572679  -116.626550     526        2003          Tecate   
...             ...          ...     ...         ...             ...   
286211    20.934640   -89.558059       9       31041         Kanasín   
286682    20.967537   -89.624392      10       31050          Mérida   
294143    23.174895  -102.868229    2180       32010       Fresnillo   
295298    22.746782  -102.518802    2279       32017       Guadalupe   
299967    22.776096  -102.571836    2427       32056       Zacatecas   

        muertes_regis_20_1  muertes_regis_20_2  muertes_resid_20_1  \
0                      251                1601                 19

In [148]:
# Importo tabla_12oct para valores geográficos
tavla = pd.read_csv(r'tabla_12oct_normalizada.csv')           # a simple vista parece que los atributos no cambian
df_tavla = pd.DataFrame(tavla)
print(df_tavla.head())

   ENTIDAD          NOM_ENT  MUN         NOM_MUN   POBTOT  identificador_145  \
0        1   Aguascalientes    1  Aguascalientes   948990                 11   
1        1   Aguascalientes    5     Jesús María   129929                 15   
2        2  Baja California    1        Ensenada   443807                 21   
3        2  Baja California    2        Mexicali  1049792                 22   
4        2  Baja California    3          Tecate   108440                 23   

   Lat_Decimal  Lon_Decimal  Altitud  identificador_2k  ...  pobreza  \
0    21.879823  -102.296047     1878                11  ...     23.7   
1    21.961273  -102.343416     1877                15  ...     26.3   
2    31.808944  -116.595134       18                21  ...     22.3   
3    32.641176  -115.475579        0                22  ...     20.2   
4    32.572679  -116.626550      526                23  ...     21.1   

   tasa_mortalidad  identificadores_00  longitud_normalizada  \
0        195.15485    

In [117]:
# Filtro la tabla_12_oct para obtener los datos de km2, población y pobreza
tavla = vert_filt(df_tavla, ['NOM_MUN', 'POBTOT', 'km2', 'pobreza', 'identificadores_00'])

# Densidad poblacional
dens_pob = den_pob(tavla)  
#print(dens_pob)

Número de atributos del dataframe: 5; cada uno con 235 datos.


In [118]:
# Compruebo que todo va bien con los identificadores 00
lista_ids = list(tavla['identificadores_00'])
acople_col(datos_agee, lista_ids, 'ids_00_comp')
# datos_agee.to_csv('datos_agee.csv', index=False) 
# Todo va bien :D

In [119]:
# Acoplo los datos de densidad poblacional
acople_col(datos_agee, dens_pob, 'dens_pobl')

In [120]:
# Agrego la población total
acople_col(datos_agee, list_pob, 'POBTOT')

In [121]:
print(datos_agee)

        Lat_Decimal  Lon_Decimal Altitud Id_mun_agee         NOM_MUN  \
0         21.879823  -102.296047    1878        1001  Aguascalientes   
1344      21.961273  -102.343416    1877        1005     Jesús María   
2904      31.808944  -116.595134      18        2001        Ensenada   
5355      32.641176  -115.475579       0        2002        Mexicali   
7664      32.572679  -116.626550     526        2003          Tecate   
...             ...          ...     ...         ...             ...   
286211    20.934640   -89.558059       9       31041         Kanasín   
286682    20.967537   -89.624392      10       31050          Mérida   
294143    23.174895  -102.868229    2180       32010       Fresnillo   
295298    22.746782  -102.518802    2279       32017       Guadalupe   
299967    22.776096  -102.571836    2427       32056       Zacatecas   

        muertes_regis_20  muertes_regis_21  muertes_resid_20  \
0                   1851              1702              1727   
1344   

In [122]:
# Obtención de tasa de mortalidad 
#tasa_mort_20 = tasa_mort(datos_agee, 'muertes_regis_20') # _regis
tasa_mort_20 = tasa_mort(datos_agee, 'muertes_resid_20')
#print(tasa_mort_20) # listo
# Obtención de tasa de mortalidad 2021
#tasa_mort_21 = tasa_mort21(datos_agee, 'muertes_regis_21')
tasa_mort_21 = tasa_mort(datos_agee, 'muertes_resid_21')
#print(tasa_mort_21) # Listo

In [123]:
# Acople tasas de mortalidad
acople_col(datos_agee, tasa_mort_20, 'tasa_mortalidad_20')
acople_col(datos_agee, tasa_mort_21, 'tasa_mortalidad_21')

In [124]:
# Agrego el índice de pobreza
pob_ind = list(tavla['pobreza'])
acople_col(datos_agee, pob_ind, 'pobreza')

In [126]:
#print(datos_agee)
datos_agee.to_csv('datos_prenormalizados.csv', index=False) 

In [135]:
################ Comenzar de aquí si no se quieren hacer cambios en los filtros
datos_pren = pd.read_csv(r'datos_prenormalizados.csv', low_memory=False)
datos_agee = pd.DataFrame(datos_pren)
print(datos_agee)

     Lat_Decimal  Lon_Decimal  Altitud  Id_mun_agee         NOM_MUN  \
0      21.879823  -102.296047     1878         1001  Aguascalientes   
1      21.961273  -102.343416     1877         1005     Jesús María   
2      31.808944  -116.595134       18         2001        Ensenada   
3      32.641176  -115.475579        0         2002        Mexicali   
4      32.572679  -116.626550      526         2003          Tecate   
..           ...          ...      ...          ...             ...   
230    20.934640   -89.558059        9        31041         Kanasín   
231    20.967537   -89.624392       10        31050          Mérida   
232    23.174895  -102.868229     2180        32010       Fresnillo   
233    22.746782  -102.518802     2279        32017       Guadalupe   
234    22.776096  -102.571836     2427        32056       Zacatecas   

     muertes_regis_20  muertes_regis_21  muertes_resid_20  muertes_resid_21  \
0                1851              1702              1727           

In [87]:
# STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP STOP
# Lo único que falta es aplicar las normalizaciones, sin embargo es necesario revisar si se quitaran algunos municipios
# de acuerdo a la incongruencia de su información, esto puede ocacionar que la normalización de valores incorrectos.

In [136]:
# Si se retiraran municipios debe hacerse aquí con un filtro horizontal

datos_finales = hor_filt(datos_agee, 'Id_mun_agee', '=', 2006) # por estado, o bien
#datos_finales = hor_filt(datos_finales, 'Id_mun_agee', '=', 7023) # Chamula Chiapas
#datos_finales = hor_filt(datos_agee, 'tasa_mortalidad', '<', 1) # muertes iguales a 0

Longitud nueva del dataframe: 234. Filtro de Id_mun_agee=2006 aplicado.


In [141]:
# Normalizo latitud, longitud, altitud, y las acoplo a la lista
lat_norm = normm(datos_finales, 'Lat_Decimal')
acople_col(datos_finales, lat_norm, 'lat_norm')
#print(lat_norm) # Funciona!

lon_norm = []
lista_lon = list(datos_finales['Lon_Decimal'].values)
max_lon = min(lista_lon)  # uso los mínimos porque están negativas todas las longitudes
for i in range(0, len(list(datos_finales['Lon_Decimal'].values))):
    valorN = lista_lon[i]/max_lon
    lon_norm.append(valorN)
acople_col(datos_finales, lon_norm, 'lon_norm')

alt_norm = normm(datos_finales, 'Altitud')
acople_col(datos_finales, alt_norm, 'alt_norm')

denpob_norm = normm(datos_finales, 'dens_pobl')
acople_col(datos_finales, denpob_norm, 'dens_pobl_norm')

tmort_norm = normm(datos_finales, 'tasa_mortalidad_20')
acople_col(datos_finales, tmort_norm, 'mortalidad_norm')

tmort_norm21 = normm(datos_finales, 'tasa_mortalidad_21')
acople_col(datos_finales, tmort_norm21, 'mortalidad_norm_21')

pobr_norm = normm(datos_finales, 'pobreza')
acople_col(datos_finales, pobr_norm, 'pobreza_norm')

In [142]:
print(datos_finales)

     Lat_Decimal  Lon_Decimal  Altitud  Id_mun_agee         NOM_MUN  \
0      21.879823  -102.296047     1878         1001  Aguascalientes   
1      21.961273  -102.343416     1877         1005     Jesús María   
2      31.808944  -116.595134       18         2001        Ensenada   
3      32.641176  -115.475579        0         2002        Mexicali   
4      32.572679  -116.626550      526         2003          Tecate   
..           ...          ...      ...          ...             ...   
230    20.934640   -89.558059        9        31041         Kanasín   
231    20.967537   -89.624392       10        31050          Mérida   
232    23.174895  -102.868229     2180        32010       Fresnillo   
233    22.746782  -102.518802     2279        32017       Guadalupe   
234    22.776096  -102.571836     2427        32056       Zacatecas   

     muertes_regis_20  muertes_regis_21  muertes_resid_20  muertes_resid_21  \
0                1851              1702              1727           

In [149]:
# Hecho lo anterior ya no se necesitan los valores no normalizados
#Data_Warehouse = vert_filt(datos_finales, ['Id_mun_agee', 'lat_norm', 'lon_norm', 'alt_norm', 'dens_pobl_norm', 'mortalidad_norm', 'pobreza_norm', 'mortalidad_norm_21'])
Data_Warehouse = vert_filt(datos_finales, ['Id_mun_agee', 'NOM_MUN','tasa_mortalidad_20','tasa_mortalidad_21', 'dens_pobl_norm', 'mortalidad_norm', 'mortalidad_norm_21', 'pobreza_norm'])
print(Data_Warehouse)

Número de atributos del dataframe: 8; cada uno con 234 datos.
     Id_mun_agee         NOM_MUN  tasa_mortalidad_20  tasa_mortalidad_21  \
0           1001  Aguascalientes           181.98295           165.22830   
1           1005     Jesús María           116.21732           109.29046   
2           2001        Ensenada           202.56553           211.12781   
3           2002        Mexicali           333.30412           192.70484   
4           2003          Tecate           189.96680           100.51641   
..           ...             ...                 ...                 ...   
230        31041         Kanasín            77.49808           109.90637   
231        31050          Mérida           154.65332           217.96169   
232        32010       Fresnillo           176.27592           129.29673   
233        32017       Guadalupe           153.49013           102.48418   
234        32056       Zacatecas           244.64096           165.76764   

     dens_pobl_norm  mort

In [150]:
# La paso a CSV para proceder con la normalización
Data_Warehouse.to_csv('Data_Warehouse.csv', index=False)

In [ ]:
# HASTA AQUÍ TERMINA EL PROCESO DE GENERACIÓN DEL DATAWAREHOUSE UTILIZANDO LOS DATOS DE CEMECE
#
#
#                           ####################################################
#                           #           ##             ##           #         ##
#                           #  #######  #######  ########  #######  #  #####  ##
#                           ##  ###############  ########  #######  #  #####  ##
#                           ###  ##############  ########  #######  #  #####  ##
#                           ####  #############  ########  #######  #         ##
#                           #####  ############  ########  #######  #  #########
#                           ######  ###########  ########  #######  #  #########
#                           #######  ##########  ########  #######  #  #########
#                           #  #####  #########  ########  #######  #  #########
#                           #          ########  ########           #  #########
#                           ####################################################
#
#
# Importo los datos de todo México para hacerlo ahora con los datos de la secretaría de salud

In [3]:
## Importo los conjuntos de datos para la mortalidad de todo el país ##
# 2020
cov2020 = pd.read_csv(r'dsCov2020_mexico.csv')           
cov2020_mex = pd.DataFrame(cov2020)
print(cov2020_mex)
# 2021
cov2021 = pd.read_csv(r'dsCov2021_mexico.csv')           
cov2021_mex = pd.DataFrame(cov2021)
#print(cov2021_mex)
# 2022
cov2022 = pd.read_csv(r'dsCov2022_mexico.csv')           
cov2022_mex = pd.DataFrame(cov2022)
#print(cov2022_mex)                                     # Todo bien hasta aquí

        ENTIDAD_RES  MUNICIPIO_RES FECHA_INGRESO   FECHA_DEF  NACIONALIDAD
0                15             95    2020-02-18  2020-05-11             1
1                 2              4    2020-06-18  2020-07-19             1
2                23              4    2020-07-20  2020-07-21             1
3                15             60    2020-12-14  2020-12-17             1
4                32             56    2020-12-15  2020-12-15             1
...             ...            ...           ...         ...           ...
210899           15             58    2020-12-17  2020-12-19             1
210900           15             25    2020-04-27  2020-04-28             1
210901            9              7    2020-12-05  2020-12-08             1
210902            9              7    2020-09-11  2020-09-12             1
210903           15             57    2020-04-29  2020-05-14             1

[210904 rows x 5 columns]


In [4]:
# Les adjunto identificadores para filtrar despúés
ids2020 = gen_id(cov2020, 'ENTIDAD_RES', 'MUNICIPIO_RES')
print(len(ids2020))
ids2021 = gen_id(cov2021, 'ENTIDAD_RES', 'MUNICIPIO_RES')
print(len(ids2021))
ids2022 = gen_id(cov2022, 'ENTIDAD_RES', 'MUNICIPIO_RES')
print(len(ids2022))

210904
181312
41150


In [5]:
# Hago los acoples
acople_col(cov2020, ids2020, 'ids2020')
acople_col(cov2021, ids2021, 'ids2021')
acople_col(cov2022, ids2022, 'ids2022')

In [25]:
# Sólo tomo los municiíos de interés
cov_mex_2020 = vert_filt2(cov2020, 'ids2020', identificadores)
cov_mex_2021 = vert_filt2(cov2021, 'ids2021', identificadores)
cov_mex_2022 = vert_filt2(cov2022, 'ids2022', identificadores)
print(cov_mex_2020)
print(cov_mex_2021)
print(cov_mex_2022)

        ENTIDAD_RES  MUNICIPIO_RES FECHA_INGRESO   FECHA_DEF  NACIONALIDAD  \
0                15             95    2020-02-18  2020-05-11             1   
1                 2              4    2020-06-18  2020-07-19             1   
2                23              4    2020-07-20  2020-07-21             1   
3                15             60    2020-12-14  2020-12-17             1   
4                32             56    2020-12-15  2020-12-15             1   
...             ...            ...           ...         ...           ...   
210899           15             58    2020-12-17  2020-12-19             1   
210900           15             25    2020-04-27  2020-04-28             1   
210901            9              7    2020-12-05  2020-12-08             1   
210902            9              7    2020-09-11  2020-09-12             1   
210903           15             57    2020-04-29  2020-05-14             1   

       ids2020  
0        15095  
1         2004  
2        230

In [26]:
# Los ordeno
cov_mex_2020.sort_values(by='ENTIDAD_RES', ascending=True, inplace=True)
cov_mex_2021.sort_values(by='ENTIDAD_RES', ascending=True, inplace=True)
cov_mex_2022.sort_values(by='ENTIDAD_RES', ascending=True, inplace=True)
# Otra forma
cov_mex_2020.sort_values(by=['ENTIDAD_RES', 'MUNICIPIO_RES'], ascending=[True, True], inplace=True)
cov_mex_2021.sort_values(by=['ENTIDAD_RES', 'MUNICIPIO_RES'], ascending=[True, True], inplace=True)
cov_mex_2022.sort_values(by=['ENTIDAD_RES', 'MUNICIPIO_RES'], ascending=[True, True], inplace=True)

C:\Users\eduar\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [59]:
#print(cov_mex_2020)
#print(cov_mex_2021)
#print(cov_mex_2022)
#print(identificadores)

In [27]:
# Hago el conteo
from collections import Counter 
def group_list(dframe, ids):   
    lst = list(dframe[ids].values) 
    return list(zip(Counter(lst).keys(), Counter(lst).values()))

In [28]:
Casos2020 = group_list(cov_mex_2020, 'ids2020')
Casos2021 = group_list(cov_mex_2021, 'ids2021')
Casos2022 = group_list(cov_mex_2022, 'ids2022')
#print(Casos2020)
print(len(Casos2020)) # Sale 233 porque hay dos municipios sin casos (2006, 7031)
print(len(Casos2021))
print(len(Casos2022))

233
232
232


In [29]:
# Veo cuáles son los que no tienen casos según los datos de secretaría de governación
# Función de listado
lis = []
for i in range(0, len(Casos2020)):
    lis.append(Casos2020[i][0])
#print(lis)
lis21 = []
for i in range(0, len(Casos2021)):
    lis21.append(Casos2021[i][0])
#print(lis21)
lis22 = []
for i in range(0, len(Casos2022)):
    lis22.append(Casos2021[i][0])
#print(lis22)

In [30]:
# listado de muertes
lism = []
for i in range(0, len(Casos2020)):
    lism.append(Casos2020[i][1])
print(lism)
lis21m = []
for i in range(0, len(Casos2021)):
    lis21m.append(Casos2021[i][1])
print(lis21m)
lis22m = []
for i in range(0, len(Casos2022)):
    lis22m.append(Casos2022[i][1])
print(lis22m)

[2020, 74, 1231, 3344, 266, 3973, 120, 591, 380, 557, 509, 414, 166, 760, 429, 93, 1803, 193, 1697, 293, 347, 167, 203, 41, 1, 64, 8, 32, 27, 60, 320, 761, 16, 224, 1721, 368, 262, 4329, 2127, 2086, 467, 4891, 1967, 6317, 621, 347, 2542, 952, 1655, 1125, 1133, 2112, 1272, 1764, 989, 793, 141, 130, 136, 101, 739, 155, 257, 856, 3239, 154, 494, 74, 212, 151, 291, 127, 1800, 48, 398, 279, 174, 155, 129, 920, 328, 327, 292, 357, 155, 3946, 270, 146, 449, 176, 142, 480, 774, 563, 1479, 177, 193, 961, 132, 723, 590, 1277, 312, 1160, 4725, 169, 377, 869, 209, 336, 615, 2343, 3459, 730, 624, 82, 1004, 111, 134, 132, 590, 2112, 2350, 267, 1316, 40, 368, 459, 1134, 463, 28, 141, 61, 476, 1147, 130, 82, 407, 185, 166, 279, 634, 98, 243, 128, 85, 598, 142, 963, 153, 50, 219, 121, 607, 54, 1539, 296, 3094, 94, 1045, 470, 111, 600, 251, 153, 91, 280, 102, 135, 4811, 223, 316, 186, 510, 168, 129, 172, 2086, 342, 326, 1775, 320, 330, 145, 2523, 432, 1161, 2497, 751, 1049, 240, 1165, 396, 2339, 284, 57

In [31]:
print(set(identificadores)-set(lis))
print(set(identificadores)-set(lis21))
print(set(identificadores)-set(lis22))

{'2006', '7031'}
{'2006', '7031', '7023'}
{'2006', '7031', '7023'}


In [ ]:
# Identifico esos registros                              Listo
# 2006: San Quintín, 7023: Chamula, 7031: Chilín         Listo
# Los elimino del dataframe original                     Listo  
# Acoplo listas de mort a los dataframe                  Listo
# Saco nuevos indices de mortalidad                      Li 
# Filtro por pertencencia de grupos                      Li

In [32]:
# Importo DataW completo
dframedw = pd.read_csv(r'Data_Warehouse_completo.csv')           
dframe_dw = pd.DataFrame(dframedw)

In [33]:
# Elimino del dataframe 2006, 7023 y 7031
# 2021
dframe2020 = hor_filt(dframe_dw, 'Id_mun_agee', '=', 2006)
dframe2020 = hor_filt(dframe2020, 'Id_mun_agee', '=', 7031)
# 2021
dframe2021 = hor_filt(dframe_dw, 'Id_mun_agee', '=', 2006)
dframe2021 = hor_filt(dframe2021, 'Id_mun_agee', '=', 7023)
dframe2021 = hor_filt(dframe2021, 'Id_mun_agee', '=', 7031)
# 2022
dframe2022 = hor_filt(dframe_dw, 'Id_mun_agee', '=', 2006)
dframe2022 = hor_filt(dframe2022, 'Id_mun_agee', '=', 7023)
dframe2022 = hor_filt(dframe2022, 'Id_mun_agee', '=', 7031)

Longitud nueva del dataframe: 234. Filtro de Id_mun_agee=2006 aplicado.
Longitud nueva del dataframe: 233. Filtro de Id_mun_agee=7031 aplicado.
Longitud nueva del dataframe: 234. Filtro de Id_mun_agee=2006 aplicado.
Longitud nueva del dataframe: 233. Filtro de Id_mun_agee=7023 aplicado.
Longitud nueva del dataframe: 232. Filtro de Id_mun_agee=7031 aplicado.
Longitud nueva del dataframe: 234. Filtro de Id_mun_agee=2006 aplicado.
Longitud nueva del dataframe: 233. Filtro de Id_mun_agee=7023 aplicado.
Longitud nueva del dataframe: 232. Filtro de Id_mun_agee=7031 aplicado.


In [34]:
# Acoplo las listas de ids de mortalidad
acople_col(dframe2020, lis, 'ids_muertes_2020')
acople_col(dframe2021, lis21, 'ids_muertes_2021')
acople_col(dframe2022, lis22, 'ids_muertes_2022')

In [35]:
# Acoplo las listas de mortalidad
acople_col(dframe2020, lism, 'muertes_2020')
acople_col(dframe2021, lis21m, 'muertes_2021')
acople_col(dframe2022, lis22m, 'muertes_2022')

In [36]:
print(dframe2020)
print(dframe2021)
print(dframe2022)

     Lat_Decimal  Lon_Decimal  Altitud  Id_mun_agee         NOM_MUN  \
0      21.879823  -102.296047     1878         1001  Aguascalientes   
1      21.961273  -102.343416     1877         1005     Jesús María   
2      31.808944  -116.595134       18         2001        Ensenada   
3      32.641176  -115.475579        0         2002        Mexicali   
4      32.572679  -116.626550      526         2003          Tecate   
..           ...          ...      ...          ...             ...   
230    20.934640   -89.558059        9        31041         Kanasín   
231    20.967537   -89.624392       10        31050          Mérida   
232    23.174895  -102.868229     2180        32010       Fresnillo   
233    22.746782  -102.518802     2279        32017       Guadalupe   
234    22.776096  -102.571836     2427        32056       Zacatecas   

     total_muertes  total_muertes_21   dens_pobl   POBTOT  tasa_mortalidad  \
0             1852              1702   805.59423   948990        195.

In [37]:
# Hasta este punto todo está bien y se tienen 3 data Warehouse para cada año con el uso de los datos de la secretaría de salud
# Paso a un csv para hacerlo en otro Script
dframe2020.to_csv('DW_SecSalud_2020.csv', index=False)
dframe2021.to_csv('DW_SecSalud_2021.csv', index=False)
dframe2022.to_csv('DW_SecSalud_2022.csv', index=False)